STEP 1 - Geração de Base de Dados Sintética Fraudulentas  
Este script cria a base de dados sintética com **10.000 registros de atendimentos médicos** que será usada para ML.  
Simula cenários reais para detecção de fraudes em planos de saúde.  

📌 Por que usar dados sintéticos?  
- O projeto tem foco em **prevenção de fraudes no contexto brasileiro**, garantindo a conformidade com a **LGPD (Lei Geral de Proteção de Dados)**, já que não utiliza dados reais de pacientes.  
- A inserção de fraudes já conhecidas (Phantom Billing, Upcoding, Duplicidade) permite **avaliar de forma controlada o desempenho do modelo de ML**.  
- Oferece uma base segura e realista para **treinar, validar e comparar algoritmos de detecção de anomalias** sem riscos éticos ou legais.  
- Apoia auditorias em saúde, reduz desperdícios financeiros e contribui para maior eficiência nos planos de saúde corporativos.  

Tipos de fraudes simuladas:  
- **Phantom Billing** → Cobrança por serviço não realizado (data de realização ausente)  
- **Upcoding** → Cobrança por procedimentos desnecessários ou de maior custo  
- **Duplicidade** → Cobrança repetida pelo mesmo serviço (valores anormalmente altos)  

Etapas principais:  
- Geração aleatória de dados realistas (pacientes, prestadores, diagnósticos, serviços, pagamentos)  
- Inserção de casos de fraude conforme porcentagens definidas  
- Salvamento da base como `'banco_dados_sintetico_operadora.csv'`  
- Download do arquivo via Google Colab  

Uso:  
- Executar no Google Colab ou em ambiente Python local  
- O CSV gerado pode ser usado para treinar e testar modelos de Machine Learning para detecção de anomalias e fraudes  
"""  

"""  
STEP 1 - Generation of Synthetic Fraudulent Dataset  
This script creates a synthetic dataset with **10,000 medical claim records** to be used for ML.  
It simulates real-world scenarios for fraud detection in health plans.  

📌 Why use synthetic data?  
- The project focuses on **fraud prevention in the Brazilian healthcare system**, ensuring compliance with the **LGPD (Brazilian General Data Protection Law)**, since no real patient data is used.  
- By injecting known fraud patterns (Phantom Billing, Upcoding, Duplicate Billing), it enables a **controlled evaluation of ML model performance**.  
- Provides a safe and realistic dataset to **train, validate, and compare anomaly detection algorithms** without ethical or legal risks.  
- Supports healthcare auditing, reduces financial waste, and improves efficiency in corporate health plans.  

Fraud types simulated:  
- **Phantom Billing** → Billing for services not performed (missing service date)  
- **Upcoding** → Billing for unnecessary or higher-cost procedures  
- **Duplicate Billing** → Repeated billing for the same service (abnormally high values)  

Main steps:  
- Random generation of realistic data (patients, providers, diagnoses, services, payments)  
- Insertion of fraud cases according to defined percentages  
- Saving the dataset as `'banco_dados_sintetico_operadora.csv'`  
- File download via Google Colab  

Usage:  
- Run in Google Colab or in a local Python environment  
- The generated CSV can be used to train and test Machine Learning models for anomaly and fraud detection  


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Parâmetros
n = 10000  # total de registros
fraude_pct = 0.10
phantom_pct = 0.0333
upcoding_pct = 0.0333
duplicidade_pct = 0.0333

# Seed para reprodutibilidade
np.random.seed(42)

# Funções auxiliares
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Gerar dados
df = pd.DataFrame({
    'id_atendimento': range(1, n + 1),
    'beneficiario_id': np.random.randint(1000, 9999, size=n),
    'prestador_id': np.random.randint(100, 999, size=n),
    'data_solicitacao': [random_date(datetime(2023, 1, 1), datetime(2024, 12, 31)) for _ in range(n)],
    'CID_': np.random.choice(['A00', 'B20', 'C34', 'D50', 'E11', 'F32', 'G40', 'H10', 'I10'], size=n),
    'status_saude': np.random.choice([-1, 0, 1], size=n, p=[0.4, 0.4, 0.2]),
    'tipo_atendimento': np.random.choice(['consulta', 'exame', 'cirurgia'], size=n, p=[0.6, 0.3, 0.1]),
    'procedimento_codigo': np.random.randint(10000, 99999, size=n),
    'autorizacao_status': np.random.choice(['aprovado', 'negado', 'pendente'], size=n, p=[0.7, 0.1, 0.2]),
    'data_realizacao': [random_date(datetime(2023, 1, 2), datetime(2025, 1, 1)) for _ in range(n)],
    'valor_pago': np.round(np.random.uniform(50, 1000, size=n), 2),
    'procedimentos_urgencia': np.random.choice(['sim', 'nao'], size=n, p=[0.2, 0.8])
})

# Phantom Billing (valor_pago > 0 e data_realizacao = NaT)
phantom_idx = df.sample(frac=phantom_pct).index
df.loc[phantom_idx, 'data_realizacao'] = pd.NaT

# Upcoding (status_saude = -1 (saudável) + procedimentos_urgencia = 'sim')
upcoding_idx = df[
    (df['status_saude'] == -1) & (df['procedimentos_urgencia'] == 'nao')
].sample(frac=upcoding_pct / 0.4).index  # ajusta para a % de saudáveis
df.loc[upcoding_idx, 'procedimentos_urgencia'] = 'sim'

# Duplicidade de cobrança (valor_pago muito alto)
duplicidade_idx = df.sample(frac=duplicidade_pct).index
df.loc[duplicidade_idx, 'valor_pago'] *= np.random.uniform(3, 5, size=len(duplicidade_idx))

# Exibir amostra
print("Amostra de dados gerados com fraudes:")
df.head()

csv_filename = "banco_dados_sintetico_operadora.csv"
df.to_csv(csv_filename, index=False)

from google.colab import files

# Baixar arquivo
files.download(csv_filename)

print("Amostra de 10 primeiros registros:")
df.head(10)

Amostra de dados gerados com fraudes:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Amostra de 10 primeiros registros:


,id_atendimento,beneficiario_id,prestador_id,data_solicitacao,CID_,status_saude,tipo_atendimento,procedimento_codigo,autorizacao_status,data_realizacao,valor_pago,procedimentos_urgencia
0,1,8270,464,2023-11-03,H10,1,exame,47030,aprovado,2023-02-14,392.490000,nao
1,2,1860,676,2024-04-30,G40,0,consulta,41792,pendente,2024-02-18,1763.374662,nao
2,3,6390,425,2023-10-08,H10,-1,consulta,17388,negado,NaT,690.640000,nao
3,4,6191,680,2023-02-26,B20,0,cirurgia,65697,aprovado,2023-04-28,793.230000,nao
4,5,6734,732,2024-11-06,C34,0,consulta,50224,pendente,2023-01-16,681.830000,nao
5,6,7265,720,2024-10-28,D50,1,exame,56959,pendente,2023-11-17,74.860000,sim
6,7,1466,927,2023-01-02,I10,0,exame,35544,negado,2024-02-12,902.000000,nao
7,8,5426,346,2024-09-27,E11,-1,cirurgia,51138,aprovado,2023-04-08,602.130000,nao
8,9,6578,533,2023-02-17,F32,-1,consulta,32698,pendente,2023-11-01,924.230000,nao
9,10,9322,192,2024-11-21,A00,0,consulta,77587,aprovado,2023-02-18,965.680000,nao
